# Part1 Data Preprocessing  

In [1]:
import pandas as pd
import os

# Part2 Crawling additional info

In [2]:
import selenium
import bs4
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import html5lib
from tqdm import tqdm

In [3]:
# 시험해볼 SIERET을 유효하게 만들어줌
def validate_siret(siret_list) : 
    valid_sirets = []
    for siret in siret_list : 
        if not len(str(siret)) == 14 : 
            # 코드 길이가 9가 아니면 그만큼 앞에 0을 붙여준다
            for _ in range(9-len(str(siret))) : 
                siret='0'+str(siret)
                
        valid_sirets.append(siret)
                     
    return valid_sirets

In [4]:
## 원하는 정보를 크롤링하여 결과값으로 데이터 프레임을 반환하는 함수 info_crawler
## bs4 사용 

def info_cralwer(siret_list) :
    
    path = "C:\\Users\\user\\chromedriver.exe"
    driver = webdriver.Chrome(path)

    # 크롬 웹 드라이버 객체 생성.
    # 크롬드라이버로 크롬을 조작.

    # 기업 정보를 조회할 프랑스 웹사이트 
    driver.get("https://annuaire-entreprises.data.gouv.fr/etablissement/24630071900118")
    #드라이버 객체에서 원하는 url로 이동.

    ###############################################################
    ######################## 기본 동작  ###########################
    driver.refresh() # 새로 고침
    # driver.quit() # 브라우저 종료
    
    
    
    기업명리스트 = []
    주소리스트 = []
    활동여부리스트 = []
    주요활동_APE리스트 = []
    법적기업형태리스트_Nature_juridique = []
    직원수리스트_Tranche_deffectif_salarié = []
    설립일리스트_Date_de_création = []

    
    for siret in siret_list : 
        try : 
            elem = driver.find_element_by_css_selector('#search-input-input')
            elem.send_keys(siret)
            elem.send_keys(Keys.RETURN)   # 엔터를 쳐서 입력
            elem = driver.find_element_by_css_selector('#page-layout > main > div > div.jsx-c751a9f3669be72c.header-section > div.jsx-c751a9f3669be72c.title > h1 > a')

            # bs4를 사용하기 위해, 현재 띄워놓은 창의 html 소스 코드 가져오기
            req = driver.page_source

            soup= BeautifulSoup(req, 'html.parser')

            elem = driver.find_element_by_xpath('//*[@id="page-layout"]/main/div/div[1]/div[1]/div/span[2]/span')
            활동여부 = elem.text

            if not soup.find_all('p')[3].text[22:27] == 'fermé' : 
            #if 활동여부 == 'en activité' :
            # 만약 공백이 하나도 없다면 
            #if not soup.find_all('i', 'jsx-63456455') : 
                # 업체명 
                기업명 = soup.find_all('a')[5].text
                info = soup.find_all('td')
                
                gap = 0 
                if len(info) > 20 : 
                    gap = len(info)-20
                
                주요활동 = info[7+gap].text.replace('\xa0copier\xa0copié!','')
                법적기업형태 = info[9+gap].text.replace('\xa0copier\xa0copié!','')     # Nature juridique
                직원수 = info[11+gap].text.replace('\xa0copier\xa0copié!','')       # Tranche effectif salarié de l’entité
                설립일 = info[13+gap].text.replace('\xa0copier\xa0copié!','')           # Date de créatio
                
                주소 = info[19+gap].text.replace('\xa0copier\xa0copié!','') 

                기업명리스트.append(기업명)
                주소리스트.append(주소)
                활동여부리스트.append(활동여부)
                주요활동_APE리스트.append(주요활동)
                법적기업형태리스트_Nature_juridique.append(법적기업형태)
                직원수리스트_Tranche_deffectif_salarié.append(직원수)
                설립일리스트_Date_de_création.append(설립일)

            else :
                next 
        
        except : next      
            
            
    result_df = pd.DataFrame(columns = ['키값', '헤브론스타국가코드', '현지언어국가명', '영문국가명', '국가언어코드값', '현지언어언어명', '영문언어명',
       '현지언어기업명', '영문기업명', '현지언어한줄소개내용', '현지언어기업소개내용', '영문한줄소개내용', '영문기업소개내용',
       '현지언어대표이사명', '영문대표이사명', '기업대표전화번호', '대표팩스번호', '대표이메일', '기업홈페이지URL',
       '페이스북URL', '인스타그램URL', '유튜브URL', '링크드인URL', '현지언어담당자명', '영문담당자명',
       '현지언어직위명', '영문직위명', '담당자부서명', '기업담당자전화번호', '담당자이메일', '담당자팩스번호',
       '기업담당자이동전화번호', '현지언어기업주소', '현지언어기업상세주소', '영문기업주소', '영문기업상세주소', '기업우편번호',
       '현지언어산업군명', '영문산업군명', '현지언어주요제품내용', '영문주요제품내용', '설립일자', '기업종업원값',
       '외감법인구분값', '매출년도값', '매출액값', '기본화폐키값', '화폐단위명', '주식시장코드', '현지언어주식시장명',
       '영문주식시장명', '상장코드', '상장일자', '기업추가정보값', '작업구분코드', '데이터생성일자', '연계처리상태코드',
       '연계처리일자'])

 
    result_df['영문국가명'] = 'FRANCE'
  
    result_df['현지언어기업명'] = 기업명리스트
    
    result_df['현지언어기업상세주소'] = 주소리스트
   
    result_df['현지언어산업군명'] = 주요활동_APE리스트
    result_df['현지언어주요제품내용'] 
    result_df['설립일자'] = 설립일리스트_Date_de_création
    result_df['기업종업원값'] 
    result_df['외감법인구분값'] = 법적기업형태리스트_Nature_juridique
    result_df['매출년도값']
    result_df['매출액값'] 
    result_df['기본화폐키값']
    result_df['화폐단위명']
    result_df['주식시장코드']
    result_df['현지언어주식시장명']
    result_df['영문주식시장명']
    result_df['상장코드']
    result_df['상장일자'] 
    result_df['기업추가정보값']
    result_df['작업구분코드']
    result_df['데이터생성일자']
    result_df['연계처리상태코드']
    result_df['연계처리일자']
    
    return result_df

In [5]:
path = "C:\\Users\\user\\chromedriver.exe"
driver = webdriver.Chrome(path)

# 크롬 웹 드라이버 객체 생성.
# 크롬드라이버로 크롬을 조작.

# 기업 정보를 조회할 프랑스 웹사이트 
driver.get("https://annuaire-entreprises.data.gouv.fr/etablissement/24630071900118")
#드라이버 객체에서 원하는 url로 이동.

###############################################################
######################## 기본 동작  ###########################
driver.refresh() # 새로 고침
# driver.quit() # 브라우저 종료



기업명리스트 = []
주소리스트 = []
활동여부리스트 = []
주요활동_APE리스트 = []
법적기업형태리스트_Nature_juridique = []
직원수리스트_Tranche_deffectif_salarié = []
설립일리스트_Date_de_création = []

In [6]:
# 기본정보가 있는 폴더 주소를 전달한다./ 
os.chdir(input('엑셀파일이 담겨있는 폴더 주소를 정확히 입력해 주세요 : '))
file_list = os.listdir()
print('아래 함수를 실행할 모든 준비가 완료되었습니다. 소요시간 : 3일')

엑셀파일이 담겨있는 폴더 주소를 정확히 입력해 주세요 : C:\Users\user\fra-doc\3~6
아래 함수를 실행할 모든 준비가 완료되었습니다. 소요시간 : 3일


In [ ]:

for file_name in tqdm(file_list[2:]) : 
    
    file = pd.read_excel(file_name)
    original_siret = file['siret']
    
    # validate_siret : siret 리스트를 유효하게 바꿔주는 함수 
    valid_siret = validate_siret(original_siret) 
    
    # info_cralwer : siret 리스트를 넣으면 해당 기업정보를 크롤링  
    result_df = info_cralwer(valid_siret) 
     
    new_file_name = file_name[:-4] + '_crawled.xlsx'
    result_df.to_excel(new_file_name, index = False)

    

 75%|█████████████████████████████████████████████████████████                   | 3/4 [25:21:09<8:04:15, 29055.91s/it]

In [4]:
# 엑셀 파일 한 건당 예상소요시간 
((65536*1.35)/60)/60

24.576000000000004

In [61]:
valid_siret_list = validate_siret(file['siret'])

In [19]:
os.chdir('C:\\Users\\user')

In [21]:
프랑스1 = pd.read_excel('프랑스1.xlsx')

In [25]:
프랑스1.columns

Index(['키값', '헤브론스타국가코드', '현지언어국가명', '영문국가명', '국가언어코드값', '현지언어언어명', '영문언어명',
       '현지언어기업명', '영문기업명', '현지언어한줄소개내용', '현지언어기업소개내용', '영문한줄소개내용', '영문기업소개내용',
       '현지언어대표이사명', '영문대표이사명', '기업대표전화번호', '대표팩스번호', '대표이메일', '기업홈페이지URL',
       '페이스북URL', '인스타그램URL', '유튜브URL', '링크드인URL', '현지언어담당자명', '영문담당자명',
       '현지언어직위명', '영문직위명', '담당자부서명', '기업담당자전화번호', '담당자이메일', '담당자팩스번호',
       '기업담당자이동전화번호', '현지언어기업주소', '현지언어기업상세주소', '영문기업주소', '영문기업상세주소', '기업우편번호',
       '현지언어산업군명', '영문산업군명', '현지언어주요제품내용', '영문주요제품내용', '설립일자', '기업종업원값',
       '외감법인구분값', '매출년도값', '매출액값', '기본화폐키값', '화폐단위명', '주식시장코드', '현지언어주식시장명',
       '영문주식시장명', '상장코드', '상장일자', '기업추가정보값', '작업구분코드', '데이터생성일자', '연계처리상태코드',
       '연계처리일자'],
      dtype='object')

In [16]:
# test code : 추출이 안 된 케이스 

path = "C:\\Users\\user\\chromedriver.exe"
driver = webdriver.Chrome(path)

# 크롬 웹 드라이버 객체 생성.
# 크롬드라이버로 크롬을 조작.

# 기업 정보를 조회할 프랑스 웹사이트 
driver.get("https://annuaire-entreprises.data.gouv.fr/etablissement/30401225500069")
#드라이버 객체에서 원하는 url로 이동.

###############################################################
######################## 기본 동작  ###########################
driver.refresh() # 새로 고침
# driver.quit() # 브라우저 종료



기업명리스트 = []
주소리스트 = []
활동여부리스트 = []
주요활동_APE리스트 = []
법적기업형태리스트_Nature_juridique = []
직원수리스트_Tranche_deffectif_salarié = []
설립일리스트_Date_de_création = []

siret = 30401225500069

elem = driver.find_element_by_css_selector('#search-input-input')
elem.send_keys(siret)
elem.send_keys(Keys.RETURN)   # 엔터를 쳐서 입력
elem = driver.find_element_by_css_selector('#page-layout > main > div > div.jsx-c751a9f3669be72c.header-section > div.jsx-c751a9f3669be72c.title > h1 > a')

# bs4를 사용하기 위해, 현재 띄워놓은 창의 html 소스 코드 가져오기
req = driver.page_source

soup= BeautifulSoup(req, 'html.parser')

elem = driver.find_element_by_xpath('//*[@id="page-layout"]/main/div/div[1]/div[1]/div/span[2]/span')
활동여부 = elem.text



if not soup.find_all('p')[3].text[22:27] == 'fermé' : 
#if 활동여부 == 'en activité' :
# 만약 공백이 하나도 없다면 
#if not soup.find_all('i', 'jsx-63456455') : 
    # 업체명 
    기업명 = soup.find_all('a')[5].text
    info = soup.find_all('td')
    
    # 아래 if 문은 만약 페이지 구성이 달라지는 경우, 올바르게 정보를 추출할 수 있도록 추가한 구문. 
    # 예를들어 siret 30401265100051, 30401225500069 의 경우 페이지 구성이 약간 달라진다. 
    if len(info) > 20 : 
        gap = len(info)-20
    
    
    주요활동 = info[7].text.replace('\xa0copier\xa0copié!','')
    법적기업형태 = info[9].text.replace('\xa0copier\xa0copié!','')     # Nature juridique
    직원수 = info[11].text.replace('\xa0copier\xa0copié!','')       # Tranche effectif salarié de l’entité
    설립일 = info[13].text.replace('\xa0copier\xa0copié!','')           # Date de créatio

    주소 = info[19].text.replace('\xa0copier\xa0copié!','') 

    기업명리스트.append(기업명)
    주소리스트.append(주소)
    활동여부리스트.append(활동여부)
    주요활동_APE리스트.append(주요활동)
    법적기업형태리스트_Nature_juridique.append(법적기업형태)
    직원수리스트_Tranche_deffectif_salarié.append(직원수)
    설립일리스트_Date_de_création.append(설립일)



In [17]:
print(기업명)
print(주소)
print(활동여부)
print(주요활동)
print(법적기업형태)
print(직원수)
print(설립일)

Euro corse voyages
Avis de situation
en activité
FR00 304 012 255
49.39B - Autres transports routiers de voyageurs
Société à responsabilité limitée (sans autre indication)
Unité non employeuse (pas de salarié au cours de l'année de référence et pas d'effectif au 31/12)


In [21]:
len(info )

22

In [22]:
# test code : 추출이 잘 된 케이스 

path = "C:\\Users\\user\\chromedriver.exe"
driver = webdriver.Chrome(path)

# 크롬 웹 드라이버 객체 생성.
# 크롬드라이버로 크롬을 조작.

# 기업 정보를 조회할 프랑스 웹사이트 
driver.get("https://annuaire-entreprises.data.gouv.fr/etablissement/30401225500069")
#드라이버 객체에서 원하는 url로 이동.

###############################################################
######################## 기본 동작  ###########################
driver.refresh() # 새로 고침
# driver.quit() # 브라우저 종료



기업명리스트 = []
주소리스트 = []
활동여부리스트 = []
주요활동_APE리스트 = []
법적기업형태리스트_Nature_juridique = []
직원수리스트_Tranche_deffectif_salarié = []
설립일리스트_Date_de_création = []

siret = 30401225500044

elem = driver.find_element_by_css_selector('#search-input-input')
elem.send_keys(siret)
elem.send_keys(Keys.RETURN)   # 엔터를 쳐서 입력
elem = driver.find_element_by_css_selector('#page-layout > main > div > div.jsx-c751a9f3669be72c.header-section > div.jsx-c751a9f3669be72c.title > h1 > a')

# bs4를 사용하기 위해, 현재 띄워놓은 창의 html 소스 코드 가져오기
req = driver.page_source

soup= BeautifulSoup(req, 'html.parser')

elem = driver.find_element_by_xpath('//*[@id="page-layout"]/main/div/div[1]/div[1]/div/span[2]/span')
활동여부 = elem.text



if not soup.find_all('p')[3].text[22:27] == 'fermé' : 
#if 활동여부 == 'en activité' :
# 만약 공백이 하나도 없다면 
#if not soup.find_all('i', 'jsx-63456455') : 
    # 업체명 
    기업명 = soup.find_all('a')[5].text
    info = soup.find_all('td')
    주요활동 = info[7].text.replace('\xa0copier\xa0copié!','')
    법적기업형태 = info[9].text.replace('\xa0copier\xa0copié!','')     # Nature juridique
    직원수 = info[11].text.replace('\xa0copier\xa0copié!','')       # Tranche effectif salarié de l’entité
    설립일 = info[13].text.replace('\xa0copier\xa0copié!','')           # Date de créatio

    주소 = info[19].text.replace('\xa0copier\xa0copié!','') 

    기업명리스트.append(기업명)
    주소리스트.append(주소)
    활동여부리스트.append(활동여부)
    주요활동_APE리스트.append(주요활동)
    법적기업형태리스트_Nature_juridique.append(법적기업형태)
    직원수리스트_Tranche_deffectif_salarié.append(직원수)
    설립일리스트_Date_de_création.append(설립일)


In [23]:
len(info)

20

In [35]:
print(기업명)
print(주소)
print(활동여부)
print(주요활동)
print(법적기업형태)
print(직원수)
print(설립일)

Euro corse voyages
1 rue jose luccioni, 20200 Bastia
en activité
49.39B - Autres transports routiers de voyageurs
Société à responsabilité limitée (sans autre indication)
Unité non employeuse (pas de salarié au cours de l'année de référence et pas d'effectif au 31/12)
30/11/2007


In [34]:
# 추출 잘 안됀 케이스 : siret 위에 두 줄이 추가돼있음 



# test code : 추출이 잘 된 케이스 

path = "C:\\Users\\user\\chromedriver.exe"
driver = webdriver.Chrome(path)

# 크롬 웹 드라이버 객체 생성.
# 크롬드라이버로 크롬을 조작.

# 기업 정보를 조회할 프랑스 웹사이트 
driver.get("https://annuaire-entreprises.data.gouv.fr/etablissement/30401225500069")
#드라이버 객체에서 원하는 url로 이동.

###############################################################
######################## 기본 동작  ###########################
driver.refresh() # 새로 고침
# driver.quit() # 브라우저 종료



기업명리스트 = []
주소리스트 = []
활동여부리스트 = []
주요활동_APE리스트 = []
법적기업형태리스트_Nature_juridique = []
직원수리스트_Tranche_deffectif_salarié = []
설립일리스트_Date_de_création = []

siret = 30401225500069

elem = driver.find_element_by_css_selector('#search-input-input')
elem.send_keys(siret)
elem.send_keys(Keys.RETURN)   # 엔터를 쳐서 입력
elem = driver.find_element_by_css_selector('#page-layout > main > div > div.jsx-c751a9f3669be72c.header-section > div.jsx-c751a9f3669be72c.title > h1 > a')

# bs4를 사용하기 위해, 현재 띄워놓은 창의 html 소스 코드 가져오기
req = driver.page_source

soup= BeautifulSoup(req, 'html.parser')

elem = driver.find_element_by_xpath('//*[@id="page-layout"]/main/div/div[1]/div[1]/div/span[2]/span')
활동여부 = elem.text



if not soup.find_all('p')[3].text[22:27] == 'fermé' : 
#if 활동여부 == 'en activité' :
# 만약 공백이 하나도 없다면 
#if not soup.find_all('i', 'jsx-63456455') : 
    # 업체명 
    기업명 = soup.find_all('a')[5].text
    info = soup.find_all('td')
    
    # 해당 if 문은 만약 페이지 구성이 달라지는 경우, 올바르게 정보를 추출할 수 있도록 추가한 구문. 
    # for example, when siret 30401265100051, 30401225500069
    if len(info) > 20 : 
        gap = len(info)-20
    
    주요활동 = info[7+gap].text.replace('\xa0copier\xa0copié!','')
    법적기업형태 = info[9+gap].text.replace('\xa0copier\xa0copié!','')     # Nature juridique
    직원수 = info[11+gap].text.replace('\xa0copier\xa0copié!','')       # Tranche effectif salarié de l’entité
    설립일 = info[13+gap].text.replace('\xa0copier\xa0copié!','')           # Date de créatio

    주소 = info[19+gap].text.replace('\xa0copier\xa0copié!','') 

    기업명리스트.append(기업명)
    주소리스트.append(주소)
    활동여부리스트.append(활동여부)
    주요활동_APE리스트.append(주요활동)
    법적기업형태리스트_Nature_juridique.append(법적기업형태)
    직원수리스트_Tranche_deffectif_salarié.append(직원수)
    설립일리스트_Date_de_création.append(설립일)
